# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0

for sqrt_value in generator:
    sum += sqrt_value

print(sum)

8.382332347441762


In [3]:
#Question 2
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [13]:
data = []
age_sum = 0

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    data.append(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    data.append(person)

for person in data:
    age_sum += person["Age"]
    
print(age_sum)

353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [4]:
%%capture
!pip install dlt
!pip install dlt[duckdb]
!pip install pandas

In [18]:
import dlt

pipeline = dlt.pipeline(pipeline_name="people_data",
                        destination='duckdb', 
                        dataset_name='people_info')

data = []

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    data.append(person)

info = pipeline.run(data, 
                    table_name="people", 
                    write_disposition="replace")

print(info)

Pipeline people_data load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:////home/forg/Documents/de_zoomcamp_2024/workshop1/people_data.duckdb location to store data
Load package 1708009083.356918 is LOADED and contains no failed jobs


In [19]:
updated_data = []

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}
        
for person in people_2():
    updated_data.append(person)

info = pipeline.run(updated_data, 
                    table_name="people", 
                    write_disposition="merge",
                    primary_key="ID")

print(info)

Pipeline people_data load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:////home/forg/Documents/de_zoomcamp_2024/workshop1/people_data.duckdb location to store data
Load package 1708009088.7336054 is LOADED and contains no failed jobs


In [22]:
# show outcome

import duckdb
import pandas as pd

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

rides = conn.sql("SELECT * FROM people ORDER BY id").df()
display(rides)

print("\n\n\n sum of ages below:")

age = conn.sql("SELECT SUM(age) FROM people").df()
display(age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708009083.356918,Ww4NAki633bwjg,None
1,2,Person_2,27,City_A,1708009083.356918,A+1tRpIRpc2+Tw,None
2,3,Person_3,33,City_B,1708009088.7336054,/viaH+F+9tP5VA,Job_3
3,4,Person_4,34,City_B,1708009088.7336054,WBMqVwZeSE3KAw,Job_4
4,5,Person_5,35,City_B,1708009088.7336054,X8c0s512B6CL+g,Job_5
5,6,Person_6,36,City_B,1708009088.7336054,C0PJo3DoK6xV2g,Job_6
6,7,Person_7,37,City_B,1708009088.7336054,aS2crj8GEjyprg,Job_7
7,8,Person_8,38,City_B,1708009088.7336054,rd+UZccqsHSxAA,Job_8





 sum of ages below:


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX